## load library

In [1]:
#python

import json
import copy
import random
import yaml

from datetime import datetime
from typing import Any, Dict, Tuple, Union, List
from tqdm import tqdm,notebook

#automl optuna
import optuna

#sklearn

from sklearn.metrics import f1_score

#numpy

import numpy as np

#pytorch

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.models as models

#special optimizer

import madgrad
from adamp import SGDP,AdamP

#baseline

import warnings

warnings.filterwarnings('ignore')

import os
import sys

sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('/opt/ml/code/src/'))))

from src.dataloader import create_dataloader
from src.loss import CustomCriterion
from src.model import Model
from src.trainer import TorchTrainer
from src.utils.common import get_label_counts, read_yaml
from src.utils.macs import calc_macs
from src.utils.torch_utils import check_runtime, model_info
from src.augmentation.policies import simple_augment_test
from src.utils.inference_utils import run_model
from src.utils.torch_utils import save_model

## hyperparameter

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## fix seed

In [3]:
#torch seed
torch.manual_seed(30)
torch.cuda.manual_seed(30)

#numpy seed
np.random.seed(30)

#python seed
random.seed(30)

## load config

In [4]:
DATA_CONFIG = read_yaml(cfg="/opt/ml/code/configs/data/taco.yaml")

## define hyperparameter search space function

In [5]:
def search_hyperparam(trial: optuna.trial.Trial) -> Dict[str, Any]:
    """Search hyperparam from user-specified search space."""
    epochs = trial.suggest_int("epochs", low=50, high=200, step=50)
    img_size = trial.suggest_categorical("img_size", [18,24,30,36,42])
    #epochs = 200
    #img_size = 96
    n_select = trial.suggest_int("n_select", low=0, high=6, step=2)
    batch_size = trial.suggest_int("batch_size", low=16, high=64, step=16)
    return {
        "EPOCHS": epochs,
        "IMG_SIZE": img_size,
        "n_select": n_select,
        "BATCH_SIZE": batch_size
    }

## load pretrained model

In [69]:
model_instance = models.shufflenet_v2_x0_5(pretrained=True)

In [70]:
model_instance.fc = nn.Linear(in_features=1024, out_features=9, bias=True)

In [71]:
model_instance

ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_

## define train function

In [6]:
def train(model_instance, model_path, optimizer, scheduler, criterion, scaler, n_epoch, train_dl, val_dl, device):
    
    #n_epoch = 30

    best_test_acc = -1.0
    best_test_f1 = -1.0

    num_classes = 9

    label_list = [i for i in range(num_classes)]

    for epoch in range(n_epoch):
        ############## prune trial ###############
        if epoch == 10:
            if best_test_f1 < 0.28:
                print(f'This model has very low f1:{best_test_f1} in 10 epochs')
                success=False
                return best_test_acc,best_test_f1,success
        if epoch == 30:
            if best_test_f1 < 0.4:
                print(f'This model has very low f1:{best_test_f1} in 30 epochs')
                success=False
                return best_test_acc,best_test_f1,success
        if epoch == 80:
            if best_test_f1 < 0.5:
                print(f'This model has very low f1:{best_test_f1} in 80 epochs')
                success=False
                return best_test_acc,best_test_f1,success
        #############################################
        running_loss, correct, total = 0.0, 0, 0
        preds, gt = [], []
        pbar = notebook.tqdm(enumerate(train_dl), total=len(train_dl))
        model_instance.train()
        for batch, (data, labels) in pbar:

            data, labels = data.to(device), labels.to(device)

            if scaler:
                with torch.cuda.amp.autocast():
                    outputs = model_instance(data)
            else:
                outputs = model_instance(data)
            outputs = torch.squeeze(outputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()

            if scaler:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()
            scheduler.step()

            _, pred = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
            preds += pred.to("cpu").tolist()
            gt += labels.to("cpu").tolist()

            running_loss += loss.item()
            pbar.update()
            pbar.set_description(
                f"Train: [{epoch + 1:03d}] "
                f"Loss: {(running_loss / (batch + 1)):.3f}, "
                f"Acc: {(correct / total) * 100:.2f}% "
                f"F1(macro): {f1_score(y_true=gt, y_pred=preds, labels=label_list, average='macro', zero_division=0):.2f}"
            )
        pbar.close()

        _, test_f1, test_acc = test(
            model=model_instance, test_dataloader=val_dl, scaler=scaler, criterion=criterion
        )
        if best_test_f1 > test_f1:
            continue
        best_test_acc = test_acc
        best_test_f1 = test_f1
        print(f"Model saved. Current best test f1: {best_test_f1:.3f}")
        save_model(
            model=model_instance,
            path=model_path,
            data=data,
            device=device,
        )
    
    
    success=True
    
    return best_test_acc,best_test_f1,success

In [7]:
@torch.no_grad()
def test(model,test_dataloader,scaler,criterion):
    """Test model.

    Args:
        test_dataloader: test data loader module which is a iterator that returns (data, labels)

    Returns:
        loss, f1, accuracy
    """

    #n_batch = _get_n_batch_from_dataloader(test_dataloader)

    running_loss = 0.0
    preds = []
    gt = []
    correct = 0
    total = 0

    num_classes = 9
    label_list = [i for i in range(num_classes)]

    pbar = notebook.tqdm(enumerate(test_dataloader), total=len(test_dataloader))
    model.to(device)
    model.eval()
    for batch, (data, labels) in pbar:
        data, labels = data.to(device), labels.to(device)

        if scaler:
            with torch.cuda.amp.autocast():
                outputs = model(data)
        else:
            outputs = model(data)
        outputs = torch.squeeze(outputs)
        running_loss += criterion(outputs, labels).item()

        _, pred = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()

        preds += pred.to("cpu").tolist()
        gt += labels.to("cpu").tolist()
        pbar.update()
        pbar.set_description(
            f" Val: {'':5} Loss: {(running_loss / (batch + 1)):.3f}, "
            f"Acc: {(correct / total) * 100:.2f}% "
            f"F1(macro): {f1_score(y_true=gt, y_pred=preds, labels=label_list, average='macro', zero_division=0):.2f}"
        )
    loss = running_loss / len(test_dataloader)
    accuracy = correct / total
    f1 = f1_score(
        y_true=gt, y_pred=preds, labels=label_list, average="macro", zero_division=0
    )
    
    return loss, f1, accuracy

## define mixup training

In [8]:
def mixup_data(x, y, alpha=1.0, use_cuda=False):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [9]:
def mixup_train(model_instance, model_path, optimizer, scheduler, criterion, scaler, n_epoch, train_dl, val_dl, device):
    
    #n_epoch = 30

    best_test_acc = -1.0
    best_test_f1 = -1.0

    num_classes = 9

    label_list = [i for i in range(num_classes)]

    for epoch in range(n_epoch):
        ############## prune trial ###############
        if epoch == 10:
            if best_test_f1 < 0.28:
                print(f'This model has very low f1:{best_test_f1} in 10 epochs')
                success=False
                return best_test_acc,best_test_f1,success
        if epoch == 30:
            if best_test_f1 < 0.4:
                print(f'This model has very low f1:{best_test_f1} in 30 epochs')
                success=False
                return best_test_acc,best_test_f1,success
        if epoch == 80:
            if best_test_f1 < 0.5:
                print(f'This model has very low f1:{best_test_f1} in 80 epochs')
                success=False
                return best_test_acc,best_test_f1,success
        #############################################
        running_loss, correct, total = 0.0, 0, 0
        preds, gt = [], []
        pbar = notebook.tqdm(enumerate(train_dl), total=len(train_dl))
        model_instance.train()
        for batch, (data, labels) in pbar:

            data,label_a,label_b,lam = mixup_data(data,labels,alpha=1.0)
        
            data, labels, label_a, label_b = data.to(device), labels.to(device), label_a.to(device), label_b.to(device)

            if scaler:
                with torch.cuda.amp.autocast():
                    outputs = model_instance(data)
            else:
                outputs = model_instance(data)
            outputs = torch.squeeze(outputs)
            #loss = criterion(outputs, labels)
            loss = criterion(outputs, label_a) * lam + criterion(outputs, label_b)* (1-lam)

            optimizer.zero_grad()

            if scaler:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()
            scheduler.step()

            _, pred = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
            preds += pred.to("cpu").tolist()
            gt += labels.to("cpu").tolist()

            running_loss += loss.item()
            pbar.update()
            pbar.set_description(
                f"Train: [{epoch + 1:03d}] "
                f"Loss: {(running_loss / (batch + 1)):.3f}, "
                f"Acc: {(correct / total) * 100:.2f}% "
                f"F1(macro): {f1_score(y_true=gt, y_pred=preds, labels=label_list, average='macro', zero_division=0):.2f}"
            )
        pbar.close()

        _, test_f1, test_acc = test(
            model=model_instance, test_dataloader=val_dl, scaler=scaler, criterion=criterion
        )
        if best_test_f1 > test_f1:
            continue
        best_test_acc = test_acc
        best_test_f1 = test_f1
        print(f"Model saved. Current best test f1: {best_test_f1:.3f}")
        save_model(
            model=model_instance,
            path=model_path,
            data=data,
            device=device,
        )
    
    
    success=True
    
    return best_test_acc,best_test_f1,success

## define cutmix training

In [10]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [11]:
def CutMix(input,target,cutmix_prob,beta=1.0):
    r = np.random.rand(1)
    if beta > 0 and r < cutmix_prob:
        lam = np.random.beta(beta, beta)
        rand_index = torch.randperm(input.size()[0]).cuda()
        target_a = target
        target_b = target[rand_index]
        bbx1, bby1, bbx2, bby2 = rand_bbox(input.size(), lam)
        input[:, :, bbx1:bbx2, bby1:bby2] = input[rand_index, :, bbx1:bbx2, bby1:bby2]
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (input.size()[-1] * input.size()[-2]))
        return input,lam,target_a,target_b,True
    else:
        lam = np.random.beta(beta, beta)
        return input,lam,target,target,False

In [12]:
def cutmix_train(model_instance, model_path, optimizer, scheduler, criterion, scaler, n_epoch, train_dl, val_dl, device):
    
    #n_epoch = 30

    best_test_acc = -1.0
    best_test_f1 = -1.0

    num_classes = 9

    label_list = [i for i in range(num_classes)]

    for epoch in range(n_epoch):
        ############## prune trial ###############
        if epoch == 10:
            if best_test_f1 < 0.28:
                print(f'This model has very low f1:{best_test_f1} in 10 epochs')
                success=False
                return best_test_acc,best_test_f1,success
        if epoch == 30:
            if best_test_f1 < 0.4:
                print(f'This model has very low f1:{best_test_f1} in 30 epochs')
                success=False
                return best_test_acc,best_test_f1,success
        if epoch == 80:
            if best_test_f1 < 0.5:
                print(f'This model has very low f1:{best_test_f1} in 80 epochs')
                success=False
                return best_test_acc,best_test_f1,success
        #############################################
        running_loss, correct, total = 0.0, 0, 0
        preds, gt = [], []
        pbar = notebook.tqdm(enumerate(train_dl), total=len(train_dl))
        model_instance.train()
        for batch, (data, labels) in pbar:
            
            data,lam,target_a,target_b,cut = CutMix(data,labels,cutmix_prob=0.5)

            data, labels = data.to(device), labels.to(device)

            if scaler:
                with torch.cuda.amp.autocast():
                    outputs = model_instance(data)
            else:
                outputs = model_instance(data)
            outputs = torch.squeeze(outputs)
            #loss = criterion(outputs, labels)
            
            if cut:
                target_a = target_a.to(device)
                target_b = target_b.to(device)
                loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b)* (1-lam)
            else:
                target_a = target_a.to(device)
                loss = criterion(outputs, target_a)

            optimizer.zero_grad()

            if scaler:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()
            scheduler.step()

            _, pred = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
            preds += pred.to("cpu").tolist()
            gt += labels.to("cpu").tolist()

            running_loss += loss.item()
            pbar.update()
            pbar.set_description(
                f"Train: [{epoch + 1:03d}] "
                f"Loss: {(running_loss / (batch + 1)):.3f}, "
                f"Acc: {(correct / total) * 100:.2f}% "
                f"F1(macro): {f1_score(y_true=gt, y_pred=preds, labels=label_list, average='macro', zero_division=0):.2f}"
            )
        pbar.close()

        _, test_f1, test_acc = test(
            model=model_instance, test_dataloader=val_dl, scaler=scaler, criterion=criterion
        )
        if best_test_f1 > test_f1:
            continue
        best_test_acc = test_acc
        best_test_f1 = test_f1
        print(f"Model saved. Current best test f1: {best_test_f1:.3f}")
        save_model(
            model=model_instance,
            path=model_path,
            data=data,
            device=device,
        )
    
    
    success=True
    
    return best_test_acc,best_test_f1,success

## define objective function

In [19]:
def objective(trial: optuna.trial.Trial, device) -> Tuple[float, int, float]:
    
    global before_macs,success
    
    data_config = copy.deepcopy(DATA_CONFIG)
    model_instance = models.shufflenet_v2_x0_5(pretrained=True)
    model_instance.fc = nn.Linear(in_features=1024, out_features=9, bias=True)

    # hyperparams: EPOCHS, IMG_SIZE, n_select, BATCH_SIZE
    hyperparams = search_hyperparam(trial)

    data_config["AUG_TRAIN_PARAMS"]["n_select"] = hyperparams["n_select"]
    data_config["BATCH_SIZE"] = hyperparams["BATCH_SIZE"]
    data_config["EPOCHS"] = hyperparams["EPOCHS"]
    data_config["IMG_SIZE"] = hyperparams["IMG_SIZE"]

    log_dir = os.path.join("exp", datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
    

    macs = calc_macs(model_instance, (3, data_config["IMG_SIZE"], data_config["IMG_SIZE"]))
    print(f"macs: {macs}")
    
    if trial.number > 1:
        if success:
            if macs > before_macs:
                print(f' trial: {trial.number}, I want to search lower macs: now {macs} >>> before {before_macs}')
                raise optuna.structs.TrialPruned()
    
    if macs>2500000.0:      ########
        print(f' trial: {trial.number}, This model has very large macs:{macs}')
        success = False
        raise optuna.structs.TrialPruned()##############
    
    #training
    
    #fixed hyperparameter
    
    model_instance = torch.nn.DataParallel(model_instance) #half tensor를 float tensor로 바꿔줌
    
    train_path = os.path.join(data_config["DATA_PATH"], "train")
    
    train_dl, val_dl, test_dl = create_dataloader(data_config)
    
    training_type = trial.suggest_categorical('training',['basic','mixup','cutmix'])
    
    loss_type = trial.suggest_categorical('loss',['softmax','weighted','customloss','logit_adjustment_loss','label_smoothing'])
    
    if training_type == 'basic':
        criterion = CustomCriterion(
            samples_per_cls=get_label_counts(train_path)
            if data_config["DATASET"] == "TACO"
            else None,
            device=device,
            loss_type=loss_type
        )
    else:
        criterion = CustomCriterion(
            samples_per_cls=get_label_counts(train_path)
            if data_config["DATASET"] == "TACO"
            else None,
            device=device,
            loss_type='label_smoothing'
        )
        
    
    # Amp loss scaler
    scaler = (
        torch.cuda.amp.GradScaler() if data_config['FP16'] and device != torch.device("cpu") else None
    )
    #scaler=None
    
    lr = trial.suggest_uniform('learning_rate',0.1,1.0)
    momentum = trial.suggest_uniform('momentum',0.0,1.0)
    #weight_decay = trial.suggest_uniform('weight_decay',1e-6,1e-1)
    #nesterov = trial.suggest_categorical('nesterov',[True,False])
    
    optimizer = torch.optim.SGD(
            model_instance.parameters(), lr=lr, momentum=momentum
        )
    
    #optimizer_type = trial.suggest_categorical('optimizer',['SGD','SGDP'])
    
    #if optimizer_type == 'SGD':
        #optimizer = torch.optim.SGD(
            #model_instance.parameters(), lr=lr, momentum=momentum
        #)
    #else:
        #optimizer = SGDP(model_instance.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)
        
    #elif optimizer_type == 'Adam':
        #optimizer = torch.optim.Adam(model_instance.parameters(), betas=(0.9,0.999), lr=lr, weight_decay=weight_decay)
    #elif optimizer_type == 'AdamP':
        #optimizer = AdamP(model_instance.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=weight_decay)
    #else:
        #optimizer= madgrad.MADGRAD(model_instance.parameters(),lr=lr,momentum=momentum)

    
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer=optimizer,
            max_lr=data_config["INIT_LR"],
            steps_per_epoch=len(train_dl),
            epochs=data_config["EPOCHS"],
            pct_start=0.05,
        )
    
    #gamma = trial.suggest_uniform('gamma',0.4,0.8)
    
    #scheduler_type = trial.suggest_categorical('scheduler',['OneCycleLR','MultistepLR'])
    
    #if scheduler_type == 'OneCycleLR':
        #scheduler = torch.optim.lr_scheduler.OneCycleLR(
            #optimizer=optimizer,
            #max_lr=data_config["INIT_LR"],
            #steps_per_epoch=len(train_dl),
            #epochs=data_config["EPOCHS"],
            #pct_start=0.05,
        #)
    #else:
        #scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=list(range(10,data_config['EPOCHS'],10)), gamma=gamma)
    

    save_path = os.path.join(log_dir, "best.pt")
    
    print(f"train {data_config['EPOCHS']} epochs this model!")
    
    os.makedirs(log_dir, exist_ok=True)
    
    with open(os.path.join(log_dir, 'data.yml'), 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False)
        
    
    if training_type == 'basic':
        _, best_f1,success = train(
            model_instance,
            save_path,
            optimizer,
            scheduler, 
            criterion, 
            scaler,
            data_config["EPOCHS"],
            train_dl, 
            val_dl, 
            device
        )
    elif training_type == 'mixup':
        _, best_f1,success = mixup_train(
            model_instance,
            save_path,
            optimizer,
            scheduler, 
            criterion, 
            scaler,
            data_config["EPOCHS"],
            train_dl, 
            val_dl, 
            device
        )
    
    else:
        _, best_f1,success = cutmix_train(
            model_instance,
            save_path,
            optimizer,
            scheduler, 
            criterion, 
            scaler,
            data_config["EPOCHS"],
            train_dl, 
            val_dl, 
            device
        )
        
    
    before_macs = macs

    return best_f1, macs

In [20]:
def tune(device, study_name= "pstage_automl"):

    sampler = optuna.samplers.MOTPESampler(n_startup_trials=20)

    study = optuna.create_study(
        directions=["maximize", "minimize"],
        study_name=study_name,
        sampler=sampler,
        load_if_exists=True
    )
    
    study.optimize(lambda trial: objective(trial, device), n_trials=100)

    pruned_trials = [
        t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED
    ]
    complete_trials = [
        t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE
    ]

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trials:")
    best_trials = study.best_trials

    ## trials that satisfies Pareto Fronts
    for tr in best_trials:
        print(f"  value1:{tr.values[0]}, value2:{tr.values[1]}")
        for key, value in tr.params.items():
            print(f"    {key}:{value}")

    return study

In [18]:
study_name="pstage_automl"

study = tune(device, study_name=study_name)

[I 2021-06-28 15:07:04,913] A new study created in memory with name: pstage_automl


macs: 650993.0
train 100 epochs this model!



Model saved. Current best test f1: 0.030



Model saved. Current best test f1: 0.030



Model saved. Current best test f1: 0.049


...


Model saved. Current best test f1: 0.049



Model saved. Current best test f1: 0.049


[I 2021-06-28 18:15:49,643] Trial 4 finished with values: [0.04930018357580195, 703217.0] and parameters: {'epochs': 200, 'img_size': 24, 'n_select': 6, 'batch_size': 16, 'training': 'cutmix', 'loss': 'label_smoothing', 'learning_rate': 0.6599563250027481, 'momentum': 0.440956415208949, 'weight_decay': 0.06282329141865249, 'nesterov': True, 'optimizer': 'SGDP'}. 



Model saved. Current best test f1: 0.049
This model has very low f1:0.04930018357580195 in 10 epochs
macs: 2100689.0
train 150 epochs this model!


KeyboardInterrupt: 